In [1]:
import unet
import cv2
import tensorflow as tf
from tensorflow.keras import losses, metrics

In [2]:
def normalize(input_image):
    input_image = tf.cast(cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB), tf.float32) / 255.0
    #input_mask -= 1
    return input_image

In [3]:
LEARNING_RATE = 1e-3
unet_model = unet.build_model(512, 512,
                              channels=3,
                              num_classes=3,
                              layer_depth=5,
                              filters_root=64,
                              padding="same"
                              )

unet.finalize_model(unet_model,
                    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
                    metrics=[metrics.SparseCategoricalAccuracy()],
                    auc=False,
                    learning_rate=LEARNING_RATE)

In [4]:
from joblib import load

data = load('data/data.pkl')

images, masks = data
images = [normalize(image) for image in images]
masks = [tf.reshape(tf.cast(mask, tf.float32), (512,512,1)) for mask in masks]
dataset = tf.data.Dataset.from_tensor_slices((images, masks))

In [5]:
#%load_ext tensorboard

In [6]:
trainer = unet.Trainer('unet_test')

trainer.fit(unet_model, dataset, epochs=1, batch_size=1)

 33/344 [=>............................] - ETA: 53:03 - loss: 0.4251 - sparse_categorical_accuracy: 0.9669 - mean_iou: 0.0120 - dice_coefficient: 0.0438